In [1]:
from src.amine.model import get_model, tokenize
from src.load.functions import load_data
from src.predict.functions import predict

/home/braymond/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load
data = load_data(sample=True)

In [3]:
# Tokenize & Model
tokenized = tokenize(data['train'], data['test'])
model = get_model(tokenized['nb_features'], tokenized['train_sequences_pad'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            5156550   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 5,217,051
Trainable params: 5,217,051
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Train
model.fit(
    tokenized['train_sequences_pad'],
    data['train']['label'].values,
    validation_split=.1,
    epochs=1,
    batch_size=128,
    verbose=True
)

Train on 180000 samples, validate on 20000 samples
Epoch 1/1
180000/180000 [==============================] - 80s 447us/step - loss: 0.4321 - acc: 0.7925 - val_loss: 0.3885 - val_acc: 0.8184


In [5]:
# Prediction on train set
pred = predict(model, tokenized['train_sequences_pad'])
labels_ok = -1 * (data['train']['label'].values * 2 - 1) == pred
accuracy = sum(labels_ok) / len(labels_ok)
print("Accuracy on train set: " + str(accuracy))

Accuracy on train set: 0.85253


In [6]:
# Prediction on test set
pred = predict(model, tokenized['test_sequences_pad'])

# Store
d = data['test'][['id']]
d['label'] = pred
csv_raw = d.to_csv(index=False, columns=['id', 'label'], header=['Id', 'Prediction'])
with open('dist/prediction.csv', 'w') as file:
    file.write(csv_raw)

/home/braymond/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
